In [3]:
print("okay")

okay


In [12]:
pip install -U langchain-community


  Using cached langchain_community-0.2.19-py3-none-any.whl.metadata (2.7 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached marshmallow-3.22.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
Using cached langchain_community-0.2.19-py3-none-any.whl (2.3 MB)
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached marshmallow-3.22.0-py3-none-any.whl (49 kB)
Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
Using cached mypy_extensions-1.0.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install --upgrade pinecone-client langchain


  Using cached pinecone_client-5.0.1-py3-none-any.whl.metadata (19 kB)
  Using cached pinecone_plugin_inference-1.1.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached pinecone_plugin_interface-0.0.7-py3-none-any.whl.metadata (1.2 kB)
Using cached pinecone_client-5.0.1-py3-none-any.whl (244 kB)
Using cached pinecone_plugin_inference-1.1.0-py3-none-any.whl (85 kB)
Using cached pinecone_plugin_interface-0.0.7-py3-none-any.whl (6.2 kB)
Note: you may need to restart the kernel to use updated packages.


In [15]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\chatu\anaconda3\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
